In [1]:
import cv2
import torch
import numpy as np

from yolo.model.yolo import YOLO, create_model
from yolo.config.config import Config
from hydra import initialize, compose
from lightning import Trainer
from yolo.utils.logging_utils import setup
from omegaconf import OmegaConf

from PIL import Image
from pathlib import Path
from torchvision import models, transforms
from deep_sort_realtime.deepsort_tracker import DeepSort
import torch_directml

device = torch_directml.device()
device_name = torch_directml.device_name(0)
print(device_name)

Radeon RX Vega 


In [2]:
def create_video_writer(video_cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer
def cfg() -> Config:
    with initialize(config_path="./yolo/config", version_base=None):
        cfg: Config = compose(config_name='config', overrides=[f"model=v9-c"])
        cfg.weight = None
    return cfg

In [5]:
#model = models.detection.fasterrcnn_resnet50_fpn(weights=models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT).to(device)
config_file = cfg()
OmegaConf.set_struct(config_file.model, False)
config_file.weight = None
model = create_model(model_cfg=config_file.model, weight_path="v9-c.pt" )
model = model.to(device)
model.eval()


[12/23/24 14:21:22] INFO     🚜 Building YOLO                                                            ]8;id=166432;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py\yolo.py]8;;\:]8;id=438043;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py#35\35]8;;\

                    INFO       🏗  Building backbone                                                      ]8;id=903248;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py\yolo.py]8;;\:]8;id=783626;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py#38\38]8;;\

                    INFO       🏗  Building neck                                                          ]8;id=998475;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py\yolo.py]8;;\:]8;id=505250;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py#38\38]8;;\

                    INFO       🏗  Building head                                                          ]8;id=314115;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py\yolo.py]8;;\:]8;id=567663;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py#38\38]8;;\

                    INFO       🏗  Building detection                                                     ]8;id=26947;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py\yolo.py]8;;\:]8;id=959494;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py#38\38]8;;\

                    INFO       🏗  Building auxiliary                                                     ]8;id=384620;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py\yolo.py]8;;\:]8;id=556942;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py#38\38]8;;\

[12/23/24 14:21:23] INFO     ✅ Success load model & weight                                             ]8;id=745059;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py\yolo.py]8;;\:]8;id=740481;file://c:\Users\vimal\Programming\Project\ObjectTracking\yolo\model\yolo.py#177\177]8;;\

YOLO(
  (model): ModuleList(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): RepNCSPELAN(
      (conv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (conv2): Sequential(
        (0): RepNCSP(
          (conv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, trac

In [3]:
deepsort = DeepSort(max_age=30,embedder='torchreid')

transform = transforms.Compose([transforms.ToTensor()])
video_path = Path("./video/macv-obj-tracking-video.mp4") 

c:\users\vimal\programming\python\deep-person-reid\torchreid\utils\tools.py:43: UserWarning: No file found at "c:\ProgramData\anaconda3\Lib\site-packages\deep_sort_realtime\embedder\weights\osnet_ain_ms_d_c_wtsonly.pth"
  warnings.warn('No file found at "{}"'.format(fpath))
c:\users\vimal\programming\python\deep-person-reid\torchreid\models\osnet_ain.py:492: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seriali

Successfully loaded imagenet pretrained weights from "C:\Users\vimal/.cache\torch\checkpoints\osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_ain_x1_0
- params: 2,193,616
- flops: 978,878,352


In [4]:
cap = cv2.VideoCapture(video_path)
frame_count = 0
object_times = {}
ret, frame = cap.read()
pil_img = Image.fromarray(frame)
img_tensor = transform(pil_img).unsqueeze(0).to(device)
print(img_tensor.shape)
with torch.no_grad():
    detections = model(img_tensor)
print(detections)
"""while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    # Preprocess the frame and detect objects using Faster R-CNN
    pil_img = Image.fromarray(frame)
    img_tensor = transform(pil_img).unsqueeze(0).to(device)

    with torch.no_grad():
        detections = model(img_tensor)

    # Get bounding boxes and scores (only keep detections with score > 0.5)
    boxes = detections[0]['boxes'].cpu().numpy()
    labels = detections[0]['labels'].cpu().numpy()
    scores = detections[0]['scores'].cpu().numpy()

    # Prepare detections for DeepSORT (format: x1, y1, width, height, score)
    detections_deepsort = []
    for box, label, score in zip(boxes, labels, scores):
        if(score > 0.5):
            x1, y1, x2, y2 = box
            width = x2 - x1
            height = y2 - y1
            detections_deepsort.append([[x1, y1, width, height], score, label])
        else:
            continue
    # Update DeepSORT tracker with the current frame's detections
    trackers = deepsort.update_tracks(detections_deepsort, frame=frame)

    # Draw bounding boxes and tracking IDs
    for track in trackers:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        print(track)
        if track_id not in object_times:
            object_times[track_id] = {"entry_frame": frame_count, "exit_frame": None}
        object_times[track_id]["exit_frame"] = frame_count  # Update exit frame every frame the object is tracked
        x1, y1, w, h = track.to_tlbr()  # DeepSORT returns bounding box as (x1, y1, x2, y2)
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Object Tracking", frame)

    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break"""


NameError: name 'video_path' is not defined

In [ ]:
# Print the duration for each object (frame range)
for obj_id, times in object_times.items():
    entry_frame = times["entry_frame"]
    exit_frame = times["exit_frame"]
    duration = exit_frame - entry_frame
    print(f"Object ID: {obj_id} appeared from frame {entry_frame} to frame {exit_frame} for {duration} frames.")

In [8]:
X = torch.rand(3,640,640, device=device)
print(X.shape)
y = model(X.unsqueeze(0))

torch.Size([3, 640, 640])


RuntimeError: input must be 4-dimensional